In [1]:
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import torch as to
import torch.nn as nn 
import torch.nn.functional as func
from torch.utils import data
from torch.utils.data import DataLoader
import random 
import time 
import torch.nn.utils.rnn as rnnutils
from sklearn.model_selection import train_test_split
import torchviz
from torchsummary import summary
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score 
from itertools import tee

Using TensorFlow backend.


In [2]:
DATASET = '/home/mbosnjak/Datasets/embedded_comments_last_100_min_20com_per_auth_w_wc_10_200_no_mbti.csv'
FOLDS = '/home/mbosnjak/Datasets/comments_last_100_min_20com_per_auth_w_wc_10_200.csv.folds.csv'
AUTHORS = '/home/mbosnjak/Datasets/author_profiles.csv'
NUMBER_OF_CLASSES = 2
STATE = 156
no_of_folds = 5

#TODO: add comments_per_author slice 

mbti_labels = ['introverted', 'intuitive', 'thinking', 'perceiving']
to.manual_seed(STATE)

In [4]:
batch_size = 64
learning_rates = [0.001, 0.0001, 0.05, 0.005, 0.000005, 0.01]
n_epochs = 25
use_GPU = True
comments_per_author = 100
cuda_device = to.device("cuda") if use_GPU else None
validation_p = .2
print_status_batch = 5

In [5]:
input_df = pd.read_csv(DATASET)
#input_df = pd.read_csv(DATASET, nrows = 2000) # TODO: remove nrows
folds_df = pd.read_csv(FOLDS, usecols =['author', 'fold'])
authors_profiles_df = pd.read_csv(AUTHORS)
authors_with_mbti = list(authors_profiles_df[authors_profiles_df.introverted.notnull()]['author'])

In [25]:
results = pd.DataFrame(data=None, columns=['mbti_trait', 'f1', 'precision', 'recall', 'fold', 'learning_rate', 'fold', 'batch_size', 'kernels_count'])

In [23]:
#results = results.append({'mbti_trait' : 'introverted', 'f1':0.5, 'precision':0.4, 'recall': 0.5, 'fold':0.4, 'learning_rate': 0.05}, ignore_index=True)

In [26]:
results

,mbti_trait,f1,precision,recall,fold,learning_rate


In [6]:
#TODO: add comment delimiter -> to recognize comments end and beginning
def merger(comments):
    return to.tensor(comments[:comments_per_author].as_matrix(columns=comments.columns[1:1025]))

def get_input_output(data_authors, target):
    data_df = input_df[input_df['author'].isin(data_authors)]
    input_x_df = data_df.groupby(['author']).apply(merger).sort_index()
    authors = list(input_x_df.index.values)
    output = authors_profiles_df[authors_profiles_df['author'].isin(authors)][target]
    return input_x_df.tolist(), output.tolist()

In [7]:
class ConvoCarpet(nn.Module):
    def __init__ (self, embedding_size = 1024, kernels_count=64, sentences_count=2):
        super(ConvoCarpet, self).__init__()
        
        self.conv_layer = nn.Conv2d(1, kernels_count, [sentences_count, embedding_size])
        self.pool_layer = nn.AdaptiveMaxPool2d((1, None))
        self.fc_layer = nn.Linear(64, NUMBER_OF_CLASSES)
        
    def forward(self, input_batch):
        conv_output = func.relu(self.conv_layer(input_batch))
        maxpool_output = self.pool_layer(conv_output)
        maxpool_output = maxpool_output.flatten(start_dim=1)
        linear_output = self.fc_layer(maxpool_output)
        #return func.log_softmax(linear_output)
        return linear_output


### FLOW

In [8]:
def data_preparation(target, fold):
    test_data_authors = list(folds_df[(folds_df['fold'] == fold) & (folds_df['author'].isin(authors_with_mbti))]['author'])
    train_data_authors = list(folds_df[(folds_df['fold'] != fold) & (folds_df['author'].isin(authors_with_mbti))]['author'])
    train_input, train_output = get_input_output(train_data_authors, target)
    test_input, test_output = get_input_output(test_data_authors, target)

    train_input, val_input, train_output, val_output = train_test_split(train_input, train_output, test_size=validation_p, random_state=STATE)
    assert len(train_input) == len(train_output)
    assert len(test_input) == len(test_output)
    assert len(val_input) == len(val_output)
    return train_input, train_output, val_input, val_output, test_input, test_output
        

def chunks(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)]  

def create_minibatches(data_X, data_y, minibatch_size, cuda_dev):
    for idx_list in chunks(range(len(data_X)), minibatch_size):
        data_X_idx = [data_X[index] for index in idx_list]
        data_y_idx = [int(data_y[index]) for index in idx_list]
        Xy = zip(data_X_idx, data_y_idx) # zip to permute the labels as well when sorting by length
        Xy = sorted(Xy, key = lambda x: len(x[0]), reverse = True)
        minibatch_X, minibatch_y = [list(x) for x in zip(*Xy)] # undo the zip thing to get separate variables for data and labels
        minibatch_lengths = to.tensor([len(x) for x in minibatch_X])
        minibatch_X = rnnutils.pad_sequence(minibatch_X, batch_first=True, padding_value = 0) 
        minibatch_X = minibatch_X.unsqueeze(1)
        minibatch_y = to.tensor(minibatch_y)
        if cuda_dev is not None:
            minibatch_lengths = minibatch_lengths.to(cuda_dev)
            minibatch_X = minibatch_X#.to(device=cuda_dev, dtype=to.float)
            minibatch_y = minibatch_y.to(device=cuda_dev, dtype=to.long)

        yield((minibatch_X, minibatch_y, minibatch_lengths))   


In [35]:
def calcMeasures(total_logits, total_true, threshold):
    total_preds = []
    sm_func = to.nn.Softmax(dim=1)
    for minibatch_logits in total_logits:
        sm_res = sm_func(minibatch_logits)
        minibatch_preds = sm_res[:,1] > threshold
        total_preds.append(minibatch_preds)
    total_preds = to.cat(total_preds)
    total_true = to.cat(total_true)
    F1 = f1_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    precision = precision_score(total_true.cpu(), total_preds.cpu())
    recall = recall_score(total_true.cpu(), total_preds.cpu())
    return F1, precision, recall

def apply_model(model, loss_func, data_loader):
    print('Starting models evaluation...')
    model.eval()
    total_loss = 0
    predicted = []
    true_output = []
    minibatch_loss = 0
    with to.no_grad():
        n_batch = 0
        for inputs, labels, input_len in data_loader:
            n_batch += 1
            minibatch_logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss_func(minibatch_logits, labels)
            
            total_loss += minibatch_loss
            predicted.append(minibatch_logits)
            true_output.append(labels)
            
    print(f"Evaluate total loss is {total_loss:.6f}")
    return total_loss, predicted, true_output

def train_model(train_loader, val_loader, models_name):
    print('Start training ...')
    best_f1_val = 0 
    best_checkpoint_filename = ''

    for learning_rate in learning_rates:
        model = ConvoCarpet()
        if use_GPU:
              model.to(cuda_device)
        loss_f = nn.CrossEntropyLoss()
        
        params_to_optimize = list(filter(lambda p: p.requires_grad, model.parameters())) # this will exclude the embeddings from optimization if their requires_grad was set to false
        optimizer = to.optim.Adam(params_to_optimize, learning_rate, amsgrad = True)
        
        for epoch in range(0, n_epochs):
            epoch_start = time.time()
            model.train()
            total_loss = 0
            n_batch = 0
            train_loader, train_loader_backup = tee(train_loader)
            for inputs, labels, input_len in train_loader_backup:
                n_batch += 1
                #print(f"Current batch is {n_batch}")

                optimizer.zero_grad()

                logits = model(inputs.to(device=cuda_device, dtype=to.float))
                minibatch_loss = loss_f(logits, labels)
                total_loss += minibatch_loss
                minibatch_loss.backward()
                optimizer.step()
                if(n_batch % print_status_batch == 0):
                    print("-----------------------")
                    print(f"Train - Epoch {epoch} - batch {n_batch}, batch loss is {minibatch_loss:.6f}, total loss is {total_loss:.6f}")
                    print("-----------------------")

            print("-----------------------")
            print(f"Finished training for epoch {epoch}")
            print("-----------------------")

            val_loader, val_loader_backup = tee(val_loader)
            val_loss, val_logits, true_val = apply_model(model, loss_f, val_loader_backup)
            val_f1_score, val_prec, val_recal = calcMeasures(val_logits, true_val, 0.5)
            print("-----------------------")
            print("-----------------------")
            print(f'Epoch end: {time.time()-epoch_start}')
            print(f'At the end of epoch {epoch} loss is: {total_loss:.5f}')
            print(f'Validation loss: {val_loss:.5f} - F1 score: {val_f1_score:.5f} - precision: {val_prec} - recall: {val_recal}')
            print("-----------------------")
            print("-----------------------")

            if (val_f1_score > best_f1_val):
                best_f1_val = val_f1_score
            # save a checkpoint
                best_checkpoint_filename = models_name + "_checkpoint-epoch-%05d-val_F1-%.3f-pre-%.3f-rec-%.3f-lr-%.3f.pt" % (epoch, val_f1_score, val_prec, val_recal, learning_rate)
                to.save({
                  "epoch" : epoch,
                  "model_state_dict" : model.state_dict(),
                  "optimizer_state_dict" : optimizer.state_dict(),
                  "train_loss" : total_loss,
                  "val_loss" : val_loss,
                  "val_F1" : val_f1_score,
                    "learning_rate" : learning_rate,
                    "val_precision" : val_prec,
                    "val_recall" : val_recal
                }, best_checkpoint_filename)        
    print("Finished with training.")
    print("Loading checkpoint that was best on validation data")
    checkpoint = to.load(best_checkpoint_filename)
    model.load_state_dict(checkpoint["model_state_dict"])
    return checkpoint, model, loss_f

In [36]:
for target in mbti_labels:
    print(f"Creating model for target: {target}")
    for fold in range(0, no_of_folds):
        model_name = "dl_carpet_"+target+f'_fold_{fold}'
        print(f'Processing fold {fold}...')
        
        train_input, train_output, val_input, val_output, test_input, test_output = data_preparation(target, fold)
        
        train_loader = create_minibatches(train_input, train_output, batch_size, cuda_device)
        val_loader = create_minibatches(val_input, val_output, batch_size, cuda_device)
        
        best_model_data, model, loss = train_model(train_loader, val_loader, model_name)
        learning_rate = best_model_data['learning_rate']
        
        print("."*15)
        print("."*15)
        print(f'Apply best model to test for {target} on fold {fold}.')
        test_loader = create_minibatches(test_input, test_output, batch_size, cuda_device)
        test_loss, test_logits, test_true = apply_model(model, loss, test_loader)
        test_f1_score, test_pred, test_recal = calcMeasures(test_logits, test_true, 0.5)
        print(f'Test loss: {test_loss:.5f} - F1 score: {test_f1_score:.2f} precision: {test_pred} recall: {test_recal}')
        with open(f'outputs_' + model_name + f'_lr_{learning_rate}_f1_test_{test_f1_score}_prec{test_pred}_recall{test_recal}', 'w+') as f:
            for pre_l, true_l in zip(test_logits, test_true):
                f.write(f'{pre_l}, {true_l}')
        print(f"Finished with training and testing model for {target} on fold {fold}.")
        results = results.append({'mbti_trait' : mbti_labels, 'f1': test_f1_score, 'precision': test_pred, 'recall': test_recal, 'fold': fold, 'learning_rate': learning_rate}, ignore_index=True)
        print("."*15)
        print("."*15)
results

Creating model for target: introverted
Processing fold 0...


/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Start training ...
-----------------------
Train - Epoch 0 - batch 5, batch loss is 0.472184, total loss is 2.868037
-----------------------
-----------------------
Train - Epoch 0 - batch 10, batch loss is 0.375658, total loss is 5.197680
-----------------------
-----------------------
Train - Epoch 0 - batch 15, batch loss is 0.588167, total loss is 8.252460
-----------------------
-----------------------
Train - Epoch 0 - batch 20, batch loss is 0.516885, total loss is 10.820596
-----------------------
-----------------------
Train - Epoch 0 - batch 25, batch loss is 0.445718, total loss is 13.445513
-----------------------
-----------------------
Train - Epoch 0 - batch 30, batch loss is 0.516567, total loss is 16.048956
-----------------------
-----------------------
Train - Epoch 0 - batch 35, batch loss is 0.487156, total loss is 18.556120
-----------------------
-----------------------
Train - Epoch 0 - batch 40, batch loss is 0.583835, total loss is 21.345938
-----------------

-----------------------
Train - Epoch 3 - batch 50, batch loss is 0.259455, total loss is 16.893362
-----------------------
-----------------------
Train - Epoch 3 - batch 55, batch loss is 0.310440, total loss is 18.462885
-----------------------
-----------------------
Train - Epoch 3 - batch 60, batch loss is 0.280635, total loss is 19.979906
-----------------------
-----------------------
Train - Epoch 3 - batch 65, batch loss is 0.272532, total loss is 21.375799
-----------------------
-----------------------
Train - Epoch 3 - batch 70, batch loss is 0.272396, total loss is 22.794592
-----------------------
-----------------------
Train - Epoch 3 - batch 75, batch loss is 0.220842, total loss is 24.175774
-----------------------
-----------------------
Train - Epoch 3 - batch 80, batch loss is 0.257021, total loss is 25.610600
-----------------------
-----------------------
Train - Epoch 3 - batch 85, batch loss is 0.197042, total loss is 26.859369
-----------------------
--------

Evaluate total loss is 20.787086
-----------------------
-----------------------
Epoch end: 1.0402796268463135
At the end of epoch 6 loss is: 13.14394
Validation loss: 20.78709 - F1 score: 0.67954 - precision: 0.8117206982543641 - recall: 0.5843806104129264
-----------------------
-----------------------
-----------------------
Train - Epoch 7 - batch 5, batch loss is 0.060389, total loss is 0.813313
-----------------------
-----------------------
Train - Epoch 7 - batch 10, batch loss is 0.047684, total loss is 1.334261
-----------------------
-----------------------
Train - Epoch 7 - batch 15, batch loss is 0.069295, total loss is 1.687017
-----------------------
-----------------------
Train - Epoch 7 - batch 20, batch loss is 0.063856, total loss is 1.975713
-----------------------
-----------------------
Train - Epoch 7 - batch 25, batch loss is 0.094604, total loss is 2.295469
-----------------------
-----------------------
Train - Epoch 7 - batch 30, batch loss is 0.102792, tota

...

Evaluate total loss is 11.822052
-----------------------
-----------------------
Epoch end: 1.0263829231262207
At the end of epoch 11 loss is: 45.07525
Validation loss: 11.82205 - F1 score: 0.88133 - precision: 0.7878359264497878 - recall: 1.0
-----------------------
-----------------------
-----------------------
Train - Epoch 12 - batch 5, batch loss is 0.434930, total loss is 2.641276
-----------------------
-----------------------
Train - Epoch 12 - batch 10, batch loss is 0.397093, total loss is 4.906465
-----------------------
-----------------------
Train - Epoch 12 - batch 15, batch loss is 0.600349, total loss is 7.633207
-----------------------
-----------------------
Train - Epoch 12 - batch 20, batch loss is 0.507244, total loss is 9.905733
-----------------------
-----------------------
Train - Epoch 12 - batch 25, batch loss is 0.465377, total loss is 12.427266
-----------------------
-----------------------
Train - Epoch 12 - batch 30, batch loss is 0.493088, total loss 

-----------------------
Train - Epoch 15 - batch 25, batch loss is 0.461108, total loss is 12.330706
-----------------------
-----------------------
Train - Epoch 15 - batch 30, batch loss is 0.489356, total loss is 14.843101
-----------------------
-----------------------
Train - Epoch 15 - batch 35, batch loss is 0.471236, total loss is 17.330503
-----------------------
-----------------------
Train - Epoch 15 - batch 40, batch loss is 0.565505, total loss is 19.980370
-----------------------
-----------------------
Train - Epoch 15 - batch 45, batch loss is 0.447503, total loss is 22.749935
-----------------------
-----------------------
Train - Epoch 15 - batch 50, batch loss is 0.528658, total loss is 25.497601
-----------------------
-----------------------
Train - Epoch 15 - batch 55, batch loss is 0.507733, total loss is 27.876169
-----------------------
-----------------------
Train - Epoch 15 - batch 60, batch loss is 0.464929, total loss is 30.334017
-----------------------


-----------------------
Train - Epoch 18 - batch 75, batch loss is 0.473633, total loss is 37.337784
-----------------------
-----------------------
Train - Epoch 18 - batch 80, batch loss is 0.405399, total loss is 39.867523
-----------------------
-----------------------
Train - Epoch 18 - batch 85, batch loss is 0.388859, total loss is 42.206608
-----------------------
-----------------------
Finished training for epoch 18
-----------------------
Starting models evaluation...
Evaluate total loss is 11.791296
-----------------------
-----------------------
Epoch end: 0.9399731159210205
At the end of epoch 18 loss is: 44.26493
Validation loss: 11.79130 - F1 score: 0.88133 - precision: 0.7878359264497878 - recall: 1.0
-----------------------
-----------------------
-----------------------
Train - Epoch 19 - batch 5, batch loss is 0.428618, total loss is 2.595838
-----------------------
-----------------------
Train - Epoch 19 - batch 10, batch loss is 0.390505, total loss is 4.826489
-

-----------------------
Train - Epoch 22 - batch 20, batch loss is 0.494111, total loss is 9.660731
-----------------------
-----------------------
Train - Epoch 22 - batch 25, batch loss is 0.452455, total loss is 12.124588
-----------------------
-----------------------
Train - Epoch 22 - batch 30, batch loss is 0.481754, total loss is 14.594264
-----------------------
-----------------------
Train - Epoch 22 - batch 35, batch loss is 0.466282, total loss is 17.039192
-----------------------
-----------------------
Train - Epoch 22 - batch 40, batch loss is 0.558080, total loss is 19.652014
-----------------------
-----------------------
Train - Epoch 22 - batch 45, batch loss is 0.441287, total loss is 22.372093
-----------------------
-----------------------
Train - Epoch 22 - batch 50, batch loss is 0.519682, total loss is 25.070820
-----------------------
-----------------------
Train - Epoch 22 - batch 55, batch loss is 0.498565, total loss is 27.415052
-----------------------
-

FileNotFoundError: [Errno 2] No such file or directory: 'dl_carpet_introverted_fold_0_checkpoint-epoch-00000-val_F1-0.881-pre-0.788-rec-1.000-lr-0.001.pt'

/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


95       0.0
197      1.0
490      1.0
545      1.0
897      1.0
        ... 
9923     1.0
10009    1.0
10029    1.0
10075    1.0
10265    0.0
Name: introverted, Length: 67, dtype: float64

['-Afrodisiac-',
 '-Automaticity',
 '-Avacyn',
 '-BigSexy-',
 '-BlitzN9ne',
 '-CrestiaBell',
 '-Kaionic-',
 '-Maxis',
 '-SagaQ-',
 '-Yaldabaoth-',
 '-Yuuki-',
 '-blahblah',
 '-dyad-',
 '-godofwine-',
 '-insert_user-',
 '-kodoku-',
 '-komorebi',
 '-nox-',
 '-reggie-',
 '-taco',
 '-tactical-throw-away',
 '-y-y-',
 '00--0--00-',
 '006ahmed',
 '0b3r0n',
 '0de',
 '0jun',
 '0nn0',
 '0pointenergy',
 '0vermind74',
 '0x43617373696F6E',
 '100xyz',
 '101312',
 '10MeV',
 '110-115-120',
 '11sies_',
 '123askingquestions',
 '137288',
 '13segruts',
 '140IQ',
 '14royals',
 '17-03',
 '17b29a',
 '1mas',
 '1nc09n170',
 '1nf1n1te',
 '1o8',
 '1tspCreamOfAwesome',
 '20mcgug',
 '2154',
 '21stcenturyschizoidf',
 '22Starter22',
 '22vortex22',
 '247sleepy',
 '25taiku',
 '280394433708491',
 '2Dijit8',
 '2_4_16_256',
 '2bassguitarist2',
 '2ndXCharm',
 '2roamunseen',
 '2yang1001',
 '3062226abcd',
 '310outof365',
 '325Gerbils',
 '3ebfan',
 '3jt',
 '409latte',
 '420ZeusNoScope',
 '420pakalolo420',
 '46makun',
 '4ndr0